p 139~156

# Chapter 4. 카운트 기반의 단어 표현
# - BoW, DTM, TF-IDF

# 1. 단어 표현 방법
- 국소 표현(Local Representation) : 이산 표현 / 해당 단어만 보고 특정값 맵핑

 ex) BoW, DTM, TF-IDF
- 분산 표현 (Distributed Representation) : 연속 표현 / 해당 단어 표현을 위해 주변 참고 -> 단어의 뉘앙스 표현 가능

 ex) Word2Vec, FastText, GloVe


# 2. Bag of Words (BoW)
2-1) Bag of Words란?
- 단어의 등장 순서 고려 X, 출현 빈도수(frequency) 기반 단어 표현(텍스트 수치화) 방법
- BoW만드는 과정 : 각 단어 정수 인코딩 (단어 집합 생성) -> 각 인덱스의 위치에, 단어 토큰의 등장 횟수 기록한 백터 생성

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 13.3 MB/s eta 0:00:00


In [4]:
# BoW 만드는 예제 1
from konlpy.tag import Okt

okt = Okt()

In [5]:
def build_bag_of_words(document):

  document = document.replace('.', '') # 입력으로 들어온 문서 document의 온점 제거
  tokenized_document = okt.morphs(document) # document 토큰화 (형태소 반환)

  word_to_index = {} # 단어와 단어의 정수인코딩 결과 저장할 딕셔너리
  bow = []           # 정수인코딩 순서대로 빈도수 저장할 리스트

  for word in tokenized_document: # 토큰 순회
    if word not in word_to_index.keys():        # word_to_index에 없는 토큰이면
      word_to_index[word] = len(word_to_index)  # word_to_index에 word_to_index의 마지막 인덱스 번호와 함께 추가
      bow.insert(len(word_to_index) - 1, 1)     # BoW <- word_to_index에 있는 토큰 수만큼 1 삽입 = BoW에 전부 기본값 1을 넣는다.
    else:                               # word_to_index에 이미 있는 토큰(재등장)이면
      index = word_to_index.get(word)   # 토큰의 인덱스 얻어서
      bow[index] = bow[index] + 1       # 해당하는 인덱스의 위치에 +1

  return word_to_index, bow

In [6]:
# 연습 - 오답노트
def building_bow(document):

  document = document.replace('.','')
  text_to_tokens = okt.morphs(document)

  word_integer = {}
  bow = []

  for word in text_to_tokens:
    if word not in word_integer.keys(): # 딕셔너리이므로 word_integer 아니고 word_integer.keys()
      word_integer[word] = len(word_integer)
      bow.insert(len(word_integer)-1, 1) # bow[len(word_integer)-1] = 1 아님 -> 배열.append(넣을 위치, 넣을 숫자) 사용
    else:
      index = word_integer[word]
      bow[index] = bow[index]+1

  return word_integer, bow

In [7]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

vocab, bow = build_bag_of_words(doc1)

print('vocabulary :', vocab)        # word_to_index = 단어 : 정수 저장
print('bag of words vector :', bow) # bow = 단어 빈도수 저장

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [8]:
# 예제 2
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [11]:
# doc1과 doc2를 합친 코퍼스에 대해 결과 확인

doc3 = doc1 + ' ' + doc2
print(doc3)
print()

vocab, bow = build_bag_of_words(doc3)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


- bow는 단어의 등장 횟수 수치화하는 텍스트 표현 방법
> 문서의 성격 판단에 사용 ex) 분류, 문서 유사도 구하는 문제

2-2) CountVectorizer 클래스로 BoW 만들기
- 사이킷런의 CountVectorizer 클래스 : 단어 빈도 count하여 vetor로 만듦, 길이가 2 이상인 문자만 토큰으로 인식

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

# 결과 : 길이가 2 이상인 문자만 토큰으로 인식하므로 i가 사라짐
# 띄어쓰기만을 기준으로 단어 자름 -> 조사 등의 이유로 BoW가 제대로 만들어지지 않음
# ex) '물가상승률과', '물가상승률은'으로 단어를 구분하므로 '물가상승률'을 인식하지 못함

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


2-3) 불용어 제거한 BoW 만들기
- BoW는 단어 빈도수를 기반으로 중요한 단어 확인하는 것
- BoW를 만들 때 불용어를 제거하는 것은, 자연어 처리의 정확도를 높이는 전처리 기법
- CountVectorizer에서 불용어 지정 가능

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords # nltk에서 제공하는 불용어 모음

In [18]:
# 1. 사용자가 정의한 불용어 사용

text = ["Family is not an important thing. It's everything."]

vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"]) # CountVectorizer에서 불용어 지정

print('bag of words vector :', vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [19]:
# 2. CountVectorizer 에서 제공하는 자체 불용어 사용
text = ["Family is not an important thing. It's everything."]

vect = CountVectorizer(stop_words="english")

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)
# is, not, an, It's, everything이 제거됨

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


In [20]:
# 3. NLTK 에서 지원하는 불용어 사용
text = ["Family is not an important thing. It's everything."]

stop_words = stopwords.words("english")

vect = CountVectorizer(stop_words=stop_words)

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)
# is, not, an, It's가 제거됨

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# 3. 문서 단어 행렬 (Document‑Term Matrix, DTM)
- DTM : 서로 다른 문서들의 BoW를 결합한 표현 방법
- 행과 열을 반대로 선택하면 TDM이라 부름
- 서로 다른 문서들을 비교할 수 있게 됨

3-1) DTM 표기법
- 다수의 문서에 대해 BoW행렬로 표현하고 부르는 용어

3-2) DTM의 한계
- 희소 표현
 - DTM은 원-핫 벡터와 같이 전체 단어 집합의 크기를 가짐
 - 코퍼스가 방대하다면 문서 벡터의 대부분이 0이 될 수 있음  (=희소 벡터, 희소 행렬)
 - 저장 공간 많이 차지, 계산 복잡도 높음 -> 전처리를 통해 단어 집합의 크기 줄이는 것이 중요

- 단순 빈도 수 기반 접근
 - 모든 문서에 자주 등장하는 '불용어를 근거로, 두 문서가 유사하다고 판단하면 안됨
 - 때문에 불용어와 중요한 단어에 가중치를 주는 방법이 고안됨 = TF-IDF

# 4. TF-IDF (Term Frequency-Inverse Document Frequency)
- DTM 내에 있는 각 단어에 대한 중요도 계산하는 방식
- DTM보다 많은 정보 고려하여 문서 비교 가능


4-1) TF-IDF (단어 빈도 - 역 문서 빈도)
- 단어 빈도와 역 문서 빈도를 사용하여 DTM 내 각 단어에 가중치 주는 방법
- 역 문서 빈도 : 문서의 빈도에 특정 식을 취함
- 유사도, 단어의 중요도 구하는 작업 등에 활용

<br/>

 (문서 d, 총 문서 수 n, 특정 단어 t에 대하여)
- TF-IDF = TF * IDF
 - 단어 빈도가 높고, 여러 문서에 나오지 않을수록 TF-IDF 높음
 - TF-IDF값과 단어의 중요도는 비례
- TF : 문서 d에서 특정 단어 t의 등장 횟수 (=DTM에 해당)
- DF : 특정 단어 t가 등장한 문서의 수
- IDF : (DF의 역수)*n -> 분모(DF)+1, log 씌우기(n이 커짐에 따라 IDF가 기하급수적으로 증가하는 것 방지)

 - 𝑖𝑑𝑓(𝑑, 𝑡) = 𝑙𝑜𝑔( 𝑛 / 1 + 𝑑𝑓(𝑡))


4-2) python으로 TF-IDF 구현

In [23]:
from math import log # IDF 계산에 log 사용
import pandas as pd

# 문서 저장
docs = [
  '먹고 싶은 사과', # 문서 1
  '먹고 싶은 바나나', # 문서 2
  '길고 노란 바나나 바나나', # 문서 3
  '저는 과일이 좋아요' # 문서 4
]

vocab = list(set(w for doc in docs for w in doc.split())) # docs의 doc 순회하며 공백 기준 split
# set() : 파이썬 내장 함수. 원소가 중복되지 않게 집합 생성
vocab.sort() # 생성된 단어장을 알파벳 순서로 정렬
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 9
['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']


In [31]:
# TF, IDF, TF-IDF 함수 구현

# 총 문서의 수
N = len(docs)

def tf(t, d):
  return d.count(t) # 문서 d에서 단어 t의 갯수 count

def idf(t):
  df = 0 # df = t가 등장한 문서 수 -> 0으로 초기화
  for doc in docs: # 문서 순회
    df += t in doc # doc 안에 t가 존재하면 df+1 (t in doc의 결과는 true or false)
  return log(N/(df+1)) # IDF 공식 구현

def tfidf(t, d):
  return tf(t,d)* idf(t) # TF-IDF = TF * IDF

In [32]:
# TF, IDF, TF-IDF 함수 구현 연습 코드 (내가 짠 코드)

N = len(docs) # 전체 문서 수

def tf2(d, t): # tf : 문서 내 단어 t의 개수
  return(d.count(t))

def idf2(t): # log(N/(t가 등장한 문서의 개수+1))
  t_d_count = 0
  for d in docs:
    if t in d:
      t_d_count += 1
  return log(N/(t_d_count+1))

def tfidf2(d, t):
  return(tf(d, t) * idf(t))

In [33]:
# TF구하기(= DTM을 데이터프레임에 저장해 출력하기)

result = [] # DFM의 모음인 TF 저장할 배열 생성

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):  # 문서 갯수만큼 반복
  result.append([]) # 해당 문서에 대한 DFM 저장할 차원(벡터) 생성
  d = docs[i]       # i번째 문서 가져오기
  for j in range(len(vocab)):   # 단어 리스트 내 단어 개수만큼 반복 (단어 순회 위해)
    t = vocab[j]                # 단어 순회하며 가져오기
    result[-1].append(tf(t, d)) # result의 마지막 행에, 문서 d 내의 단어 t 카운트 한 결과 append

tf_ = pd.DataFrame(result, columns = vocab) # DataFrame 형태로 출력

In [34]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [28]:
# IDF 구하기
result = []
for j in range(len(vocab)): # 단어 리스트 길이만큼 반복 (단어 리스트 순회 위해)
    t = vocab[j] # 단어 가져와
    result.append(idf(t)) # 단어가 등장한 문서 수의 역수 계산해 배열에 append

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"]) # IDF 출력
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [29]:
# TF-IDF 행렬 출력
result = []
for i in range(N): # 문서 수만큼 반복하며
  result.append([]) # 가져온 문서의 TF-IDF 저장할 차원 생성
  d = docs[i] # 문서 가져오기
  for j in range(len(vocab)): # 단어 수만큼 반복
    t = vocab[j] # 단어 가져와
    result[-1].append(tfidf(t,d)) # 마지막 행에 TF-IDF 계산 결과 append

tfidf_ = pd.DataFrame(result, columns = vocab) # dataframe 형태로 출력
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


- 실제 구현에서는 위 식과 조금 다른 식을 사용해 구현 <- 위 기본 식의 문제점 때문
- 문제점 : 분자(문서 수 n)와 분모 (df+1)의 값이 같아지면 약분되어 1이 되고, log1 = 0이 되어 가중치의 역할을 수행하지 못하게 되는 경우가 있음

4-3) 사이킷런을 이용한 DTM, TF-IDF 실습
- 사이킷런에서도 조금 변형된 TF-IDF 식을 사용하고 있음
- DTM 생성 : BoW에서 언급한 CountVectorizer를 사용
- TF-IDF 생성 : TfidfVectorizer -> TF-IDF 기본 식에서 로그의 분자항+1, 로그항+1, TF-IDF에 L2정규화한 변형 식 사용

In [35]:
# DTM 생성

from sklearn.feature_extraction.text import CountVectorizer # 문서 내 단어의 빈도수를 행렬로 생성

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray()) # 각 단어의 빈도수 arry로 저장 = DTM

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

# 첫번째 열 : 코퍼스 내 'do'의 빈도
# 두번째 열 : 코퍼스 내 'know'의 빈도

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [36]:
# TF-IDF 생성

from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer : text 내 모든 단어를 BoW로 구성 -> TF-IDF 계산 -> 각 단어의 인덱스 위치에 저장한 벡터 생성

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',
]

tfidfv = TfidfVectorizer().fit(corpus) # corpus에 대해 TF-IDF 계산, 모델 피팅(단어:정수 형태) -> 모델로 텍스트 데이터 변환 가능
print(tfidfv.transform(corpus).toarray()) # TF-IDF(문서 내 단어의 중요도) 계산 결과를 array로 출력

# 1행 1열의 의미 : 1번째 문서('you know I want your love',)에서 0번 인코딩 단어(do)의 중요성
# 1행 2열의 의미 : 1번째 문서에서 1번 인코딩 단어 (know)의 중요성
print(tfidfv.vocabulary_)


[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
